# 6th_homework

### Import Section

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

### Global Settings Section

In [2]:
random_state_global = 0

## 1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [3]:
list_columns_name = [
    'variance',
    'skewness',
    'curtosis',
    'entropy',
    'forgery'
]

df_data = pd.read_csv('data_banknote_authentication.txt', names=list_columns_name)

In [4]:
df_data.head()

,variance,skewness,curtosis,entropy,forgery
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


## 2. сделать feature engineering

##### Для решения задачи планируется использовать деревья решений, поэтому предобработка не потребуется.

## 3. обучить любой классификатор (какой вам нравится)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_data.iloc[:, :-1], df_data.iloc[:, -1],
                                                   test_size=0.5, random_state=random_state_global,
                                                   shuffle=df_data.iloc[:, -1])

In [6]:
%%time

dtc = DecisionTreeClassifier(random_state=random_state_global)

dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       385
           1       0.98      0.96      0.97       301

    accuracy                           0.97       686
   macro avg       0.97      0.97      0.97       686
weighted avg       0.97      0.97      0.97       686

Wall time: 5.97 ms


In [7]:
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, dtc.predict_proba(X_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics = pd.DataFrame(data={
    'threshold': list_threshold[index_best],
    'precision': list_precision[index_best],
    'recall': list_recall[index_best],
    'f1-score': list_f1_score[index_best]
}, index=['Base'])

df_metrics

,threshold,precision,recall,f1-score
Base,1.0,0.976351,0.960133,0.968174


## 4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

##### Используем половину Positives и столько же Unlabeled для обучения модели.

In [8]:
X = df_data.iloc[:, :-1]
y = df_data.iloc[:, -1]

X_positives = X[y == 1].sample(frac=0.5, replace=False)
X_unlabeled = X.iloc[sorted(set(X.index) - set(X_positives.index)), :].sample(n=X_positives.shape[0], replace=False)

X_train = pd.concat([X_positives, X_unlabeled])
y_train = np.zeros(X_train.shape[0])
y_train[:X_positives.shape[0]] = 1

##### Остальные объекты отложим для валидации.

In [9]:
X_test = X.iloc[sorted(set(X.index) - set(X_positives.index) - set(X_unlabeled.index)), :]
y_test = y[X_test.index.values]

## 5. применить random negative sampling для построения классификатора в новых условиях

In [10]:
%%time

dtc = DecisionTreeClassifier(random_state=random_state_global)

dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94       534
           1       0.98      0.73      0.84       228

    accuracy                           0.91       762
   macro avg       0.94      0.86      0.89       762
weighted avg       0.92      0.91      0.91       762

Wall time: 6 ms


## 6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [11]:
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, dtc.predict_proba(X_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics.loc['Random Negative Sampling', :] = [
    list_threshold[index_best],
    list_precision[index_best],
    list_recall[index_best],
    list_f1_score[index_best]
]

df_metrics

,threshold,precision,recall,f1-score
Base,1.0,0.976351,0.960133,0.968174
Random Negative Sampling,1.0,0.976608,0.732456,0.837093


##### Модель, построенная классическим методом, показала лучшие метрики качества, так как использовала больше объектов для обучения, а также была обучена на достоверных Negative объектах.

##### Однако модель, постоенная при помощи метода "Random Negative Sampling", продемонстрировала хорошие результаты с учётом обучения на Unlabeled объектах.

## 7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

### Уменьшение доли Positives

In [12]:
%%time

X = df_data.iloc[:, :-1]
y = df_data.iloc[:, -1]

X_positives = X[y == 1].sample(frac=0.3, replace=False)
X_unlabeled = X.iloc[sorted(set(X.index) - set(X_positives.index)), :].sample(n=X_positives.shape[0], replace=False)

X_train = pd.concat([X_positives, X_unlabeled])
y_train = np.zeros(X_train.shape[0])
y_train[:X_positives.shape[0]] = 1

X_test = X.iloc[sorted(set(X.index) - set(X_positives.index) - set(X_unlabeled.index)), :]
y_test = y[X_test.index.values]

dtc = DecisionTreeClassifier(random_state=random_state_global)

dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       653
           1       0.87      0.72      0.79       353

    accuracy                           0.86      1006
   macro avg       0.87      0.83      0.84      1006
weighted avg       0.87      0.86      0.86      1006

Wall time: 7.98 ms


In [13]:
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, dtc.predict_proba(X_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics.loc['Random Negative Sampling -', :] = [
    list_threshold[index_best],
    list_precision[index_best],
    list_recall[index_best],
    list_f1_score[index_best]
]

df_metrics

,threshold,precision,recall,f1-score
Base,1.0,0.976351,0.960133,0.968174
Random Negative Sampling,1.0,0.976608,0.732456,0.837093
Random Negative Sampling -,1.0,0.870307,0.722380,0.789474


### Увеличение доли Positives

In [14]:
%%time

X = df_data.iloc[:, :-1]
y = df_data.iloc[:, -1]

X_positives = X[y == 1].sample(frac=0.7, replace=False)
X_unlabeled = X.iloc[sorted(set(X.index) - set(X_positives.index)), :].sample(n=X_positives.shape[0], replace=False)

X_train = pd.concat([X_positives, X_unlabeled])
y_train = np.zeros(X_train.shape[0])
y_train[:X_positives.shape[0]] = 1

X_test = X.iloc[sorted(set(X.index) - set(X_positives.index) - set(X_unlabeled.index)), :]
y_test = y[X_test.index.values]

dtc = DecisionTreeClassifier(random_state=random_state_global)

dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       416
           1       0.94      0.78      0.86       102

    accuracy                           0.95       518
   macro avg       0.95      0.89      0.91       518
weighted avg       0.95      0.95      0.95       518

Wall time: 7.98 ms


In [15]:
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, dtc.predict_proba(X_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics.loc['Random Negative Sampling +', :] = [
    list_threshold[index_best],
    list_precision[index_best],
    list_recall[index_best],
    list_f1_score[index_best]
]

df_metrics

,threshold,precision,recall,f1-score
Base,1.0,0.976351,0.960133,0.968174
Random Negative Sampling,1.0,0.976608,0.732456,0.837093
Random Negative Sampling -,1.0,0.870307,0.722380,0.789474
Random Negative Sampling +,1.0,0.941176,0.784314,0.855615


##### Увеличение доли Positive объетков благоприятно влияет на качество построенной модели, так как у модели оказывается больше объектов для обучения. Однако при этом сокращается Out-of-The-Bag подвыборка для валидации, что снижает достоверность полученных метрик.